## Phase 0: Le Sandbox Poétique



### Chargement du modèle


In [1]:
import sys
from pathlib import Path
from llama_cpp import Llama

# Chemin vers le modèle téléchargé
MODEL_PATH = "./models/Llama-3.2-1B-Instruct.Q4_K_M.gguf"

# Vérifier que le modèle existe
if not Path(MODEL_PATH).exists():
    print(f"erreur: Le modèle n'a pas été trouvé à {MODEL_PATH}")
    print("run python download_model.py")
    sys.exit(1)


# n_ctx=2048 signifie que le modèle peut se souvenir d'environ 1500 mots de conversation
llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=2048,
    verbose=False  # True : voir les statistiques de vitesse
)

print("done")


llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64

done


### Sandbox

In [2]:
TOPIC = "le silence de l'hiver"
FORM = "sonnet"  # "sonnet", "limerick", "poème en vers libres"

prompt_structure = [
    {"role": "system", "content": "You are a skilled poet."},
    {"role": "user", "content": f"Write a {FORM} about {TOPIC}."}
]

print(f"\n--- Génération d'un {FORM} sur '{TOPIC}' ---\n")

output = llm.create_chat_completion(
    messages=prompt_structure,
    max_tokens=1000,   
    temperature=0.8,
    top_p=0.95,
)

poem = output["choices"][0]["message"]["content"]

print(poem)



--- Génération d'un sonnet sur 'le silence de l'hiver' ---

Amidst the silence of the winter's night,
When snowflakes dance, and frosty mists unfold,
The hibernating world, in icy light,
Lies slumbering, lost, in frozen fold.

The wind, a mournful sigh, whispers low,
Through trees, that creak and groan, in frozen sleep,
Their branches swayed, like skeletal foes,
As winter's grip, on earth, does keep.

The earth, in stillness, lies, frozen deep,
And in that silence, fears arise,
The trees, like sentinels, stand watch and keep,
Their vigil, through the icy winter's guise.

And I, in awe, of this winter's peace,
Am bound to silence, and the icy breeze.


## Sauvegarder le poème 


In [4]:
from pathlib import Path
from datetime import datetime

output_file = Path("poem.txt")
output_file.write_text(poem, encoding="utf-8")



597